#### Current in-product affiliations

In [1]:
from google.cloud import bigquery
import pyodbc
import pandas as pd
from datetime import datetime

In [2]:
bq_conn = bigquery.Client()
ssms_conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=Database5;DATABASE=DefinitiveHC;Trusted_Connection=yes;')

In [3]:
# Truncate BQ tables 
bq_conn.query("""DELETE FROM DS_WORK.CURRENT_PHYSICIAN_HOSPITAL_AFFILIATIONS WHERE 1=1""").result()
bq_conn.query("""DELETE FROM DS_WORK.CURRENT_PHYSICIAN_PG_AFFILIATIONS WHERE 1=1""").result()
bq_conn.query("""DELETE FROM DS_WORK.CURRENT_PHYSICIAN_ASC_AFFILIATIONS WHERE 1=1""").result()
bq_conn.query("""DELETE FROM DS_WORK.CURRENT_PHYSICIAN_IC_AFFILIATIONS WHERE 1=1""").result()

In [4]:
#Re-populate BQ tables

Hospitals

In [5]:
hosp_aff_sql = """
SELECT A.NPI, A.PRIMARY_AFFILIATION_HOSPITAL_ID AS HOSPITAL_ID, AC.HOSPITAL_NAME
, AC.FIRM_TYPE, AC.HQ_CITY
, AC.HQ_STATE
, 1 AS AFFILIATION_RANK
, CAST(GETDATE() AS DATE) AS UPDATE_DATE
FROM HOSPITAL.PHYSICIAN_COMPARE_PHYSICIANS A
JOIN HOSPITAL.ALL_COMPANIES AC 
ON A.PRIMARY_AFFILIATION_HOSPITAL_ID = AC.HOSPITAL_ID
UNION
SELECT A.NPI, A.HOSPITAL_ID2 AS HOSPITAL_ID, AC.HOSPITAL_NAME
, AC.FIRM_TYPE, AC.HQ_CITY
, AC.HQ_STATE
, 2 AS AFFILIATION_RANK
, CAST(GETDATE() AS DATE) AS UPDATE_DATE
FROM HOSPITAL.PHYSICIAN_COMPARE_PHYSICIANS A
JOIN HOSPITAL.ALL_COMPANIES AC 
ON A.HOSPITAL_ID2 = AC.HOSPITAL_ID
UNION
SELECT A.NPI, A.HOSPITAL_ID3 AS HOSPITAL_ID, AC.HOSPITAL_NAME
, AC.FIRM_TYPE, AC.HQ_CITY
, AC.HQ_STATE
, 3 AS AFFILIATION_RANK
, CAST(GETDATE() AS DATE) AS UPDATE_DATE
FROM HOSPITAL.PHYSICIAN_COMPARE_PHYSICIANS A
JOIN HOSPITAL.ALL_COMPANIES AC 
ON A.HOSPITAL_ID3 = AC.HOSPITAL_ID
UNION
SELECT A.NPI, A.HOSPITAL_ID4 AS HOSPITAL_ID, AC.HOSPITAL_NAME
, AC.FIRM_TYPE, AC.HQ_CITY
, AC.HQ_STATE
, 4 AS AFFILIATION_RANK
, CAST(GETDATE() AS DATE) AS UPDATE_DATE
FROM HOSPITAL.PHYSICIAN_COMPARE_PHYSICIANS A
JOIN HOSPITAL.ALL_COMPANIES AC 
ON A.HOSPITAL_ID4 = AC.HOSPITAL_ID
UNION
SELECT A.NPI, A.HOSPITAL_ID5 AS HOSPITAL_ID, AC.HOSPITAL_NAME
, AC.FIRM_TYPE, AC.HQ_CITY
, AC.HQ_STATE
, 5 AS AFFILIATION_RANK
, CAST(GETDATE() AS DATE) AS UPDATE_DATE
FROM HOSPITAL.PHYSICIAN_COMPARE_PHYSICIANS A
JOIN HOSPITAL.ALL_COMPANIES AC 
ON A.HOSPITAL_ID5 = AC.HOSPITAL_ID"""

In [6]:
hosp_aff_df = pd.read_sql(hosp_aff_sql, ssms_conn)

In [7]:
bq_conn.load_table_from_dataframe(hosp_aff_df, 'DS_WORK.CURRENT_PHYSICIAN_HOSPITAL_AFFILIATIONS').result()

/usr/local/lib/python3.6/site-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'HOSPITAL_NAME'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))


Physician Groups

In [8]:
pg_aff_sql = """
SELECT A.NPI, A.HOSPITAL_ID, AC.HOSPITAL_NAME, AC.FIRM_TYPE, AC.HQ_CITY, AC.HQ_STATE, CAST(GETDATE() AS DATE) AS UPDATE_DATE
FROM HOSPITAL.PHYSICIAN_COMPARE_PHYSICIANS_TO_GROUP_PRACTICE A 
JOIN HOSPITAL.PHYSICIAN_COMPARE_PHYSICIANS B 
ON A.NPI = B.NPI
JOIN HOSPITAL.ALL_COMPANIES AC
ON A.HOSPITAL_ID = AC.HOSPITAL_ID"""

In [9]:
pg_aff_df = pd.read_sql(pg_aff_sql, ssms_conn)

In [10]:
bq_conn.load_table_from_dataframe(pg_aff_df, 'DS_WORK.CURRENT_PHYSICIAN_PG_AFFILIATIONS').result()

Surgery Centers

In [11]:
asc_aff_sql = """
SELECT A.NPI, D.HOSPITAL_ID, AC.HOSPITAL_NAME, AC.FIRM_TYPE, AC.HQ_CITY, AC.HQ_STATE, CAST(GETDATE() AS DATE) AS UPDATE_DATE 
FROM HOSPITAL.PHYSICIAN_COMPARE_PHYSICIANS A
JOIN HOSPITAL.PHYSICIAN_COMPARE_PHYSICIANS_TO_GROUP_PRACTICE B
ON A.NPI=B.NPI
JOIN HOSPITAL.PHYSICIAN_GROUPS C
ON B.HOSPITAL_ID=C.HOSPITAL_ID
JOIN HOSPITAL.AMBULATORY_SURGERY_CENTERS D
ON B.HOSPITAL_ID=D.PG_PARENT_ID
JOIN HOSPITAL.ALL_COMPANIES AC
ON D.HOSPITAL_ID = AC.HOSPITAL_ID
"""

In [12]:
asc_aff_df = pd.read_sql(asc_aff_sql, ssms_conn)

In [13]:
bq_conn.load_table_from_dataframe(asc_aff_df, 'DS_WORK.CURRENT_PHYSICIAN_ASC_AFFILIATIONS').result()

Imaging Centers

In [14]:
ic_aff_sql = """
SELECT A.NPI, D.HOSPITAL_ID, AC.HOSPITAL_NAME, AC.FIRM_TYPE, AC.HQ_CITY, AC.HQ_STATE, CAST(GETDATE() AS DATE) AS UPDATE_DATE 
FROM HOSPITAL.PHYSICIAN_COMPARE_PHYSICIANS A
JOIN HOSPITAL.PHYSICIAN_COMPARE_PHYSICIANS_TO_GROUP_PRACTICE B
ON  A.NPI=B.NPI 
JOIN HOSPITAL.PHYSICIAN_GROUPS C
ON B.HOSPITAL_ID=C.HOSPITAL_ID
JOIN HOSPITAL.IMAGING D
ON B.HOSPITAL_ID=D.PG_PARENT_ID
JOIN HOSPITAL.ALL_COMPANIES AC
ON D.HOSPITAL_ID = AC.HOSPITAL_ID
"""

In [15]:
ic_aff_df = pd.read_sql(ic_aff_sql, ssms_conn)

In [16]:
bq_conn.load_table_from_dataframe(ic_aff_df, 'DS_WORK.CURRENT_PHYSICIAN_IC_AFFILIATIONS').result()